<h1>
    <a>
    AWS Sagemaker - Part 1:
    </a>
</h1>

<h5> By: Keshav Likhar; Date: Aug 14,2022</h5>

<h2>
    <a>
    Goals:
    </a>
</h2>


- Creating S3 Bucket
- Mapping train and Test Data in S3
- Mapping the path of the models in S3

In [15]:
# importing required libraries

import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session

import pandas as pd
import urllib
import numpy as np

In [8]:
bucketname = 'keshavbankapplication'
my_region = boto3.session.Session().region_name
print(my_region)

us-east-2


In [9]:
s3_obj = boto3.resource('s3')
try:
    if my_region == 'us-east-2':
        s3_obj.create_bucket(Bucket=bucketname,
                         CreateBucketConfiguration={'LocationConstraint': 'us-east-2'})
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


In [11]:
# set an output path where the trained model will be saved
prefix = 'xgboost-as-a-built-in-algo'
output_path ='s3://{}/{}/output'.format(bucketname, prefix)
print(output_path)

s3://keshavbankapplication/xgboost-as-a-built-in-algo/output


#### Downloading the dataset And Storing in S3

In [13]:
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [16]:
### Train Test split

train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [18]:
### Saving Train And Test Into Buckets
## We start with Train Data

import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)

boto3.Session().resource('s3').Bucket(bucketname).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')

s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucketname, prefix), content_type='csv')

In [21]:
# Test Data Into Buckets
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucketname).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucketname, prefix), content_type='csv')